Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under he License.
the License.

## Imports and Parameters

In [ ]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time
import pandas as pd
import numpy as np
from autocorrect import Speller
import re

pd.set_option('display.max_colwidth', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model

In [ ]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q22'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ22ANCES'
CODE_TABLE = 'dbo.AQ22ANCES_Codes'
CLOSED_TABLE = 'dbo.Q22ANCESMultiResponse'
RESULTS_TABLE = 'dbo.AQ22ANCES_RESULTS'

# which column to use/create 
RESPONSE_COLUMN = 'aq22_cleaned'
OUTPUT_COLUMNS = 'q22ances_c'
N_COLUMNS = 5

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# amount of synthetic data to produce
N_PER_CATEGORY = 10_000
N_SYNTHETIC_MIXED = 200_000

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Read from Database

In [ ]:
# Read in all data required to build model
connection = connect.create_connection(CRED_PATH)

# actual responses
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

# codes to match
code_df = connect.fetch_table(CODE_TABLE, connection)

# closed respones (for multi response frequencies)
df_closed = connect.fetch_table(CLOSED_TABLE, connection)

In [ ]:
df_open.head()

In [ ]:
# Reshaping the dataframe
df_reshaped = preprocess.reshape_df(df_open)

In [ ]:
# filter to only cycle 1 for now 
df_reshaped[df_reshaped.cycle == 1].head()

## Preprocess Data

* Use only cycle 1 for training.
* Use the cleaned column.
* Correct spelling where possible (takes longer time to run).
* Create input and output tables for model (input table takes a while to produce)

In [ ]:
# for training purposes, open responses should only be those from cycle 1
df = df_reshaped[df_reshaped.cycle == 1].reset_index(drop=True)
df.head()

In [ ]:
# get long form table of codes: step 1: turn countries into rows
code_df_long_tmp = preprocess.get_long_form_codes(code_df)
code_df_long_tmp.head(20)

In [ ]:
# step 2: add additional rows for each country (nationalities, other spellings, etc.)
code_df_long = preprocess.get_long_form_codes_q22(code_df_long_tmp)

In [ ]:
code_df_long.head()

In [ ]:
code_list = code_df_long.description.values

# create a spell checker
spell = Speller()

# get a list of all words that are directly in the code words 
all_words = []
for word in code_list:
    words = re.split(r'\sand\s|[,;()/\r\n\s]+', word)
    for x in words:
        if len(x) > 0:
            all_words.append(x)
            
# add additional words that are not inaccurate 
words = [
    'Inuit',
    'Wsanec',
    'Tongo',
    'Levant',
    'Berber',
    'Guinea-Bissau',
    'Goan',
    'Dessie',
    'Chilean',
    'Burundi',
    'Burmese,
    'Hongkonger'
]

for word in all_words + words:

    for x in [word, word.upper(), word.lower()]:
        if x in spell.nlp_data:
            continue
            
        spell.nlp_data[word] = 100
        spell.nlp_data[word.upper()] = 100
        spell.nlp_data[word.lower()] = 100

In [ ]:
# spellcheck responses 
df[RESPONSE_COLUMN] = df.aq22ances.apply(lambda x: preprocess.correct_spelling(x, spell=spell))
df.head()

In [ ]:
reload(preprocess)

In [ ]:
# build a training dataset for the model 
# first part of dataset: actual data
code_df_long['code'] = code_df_long['code'].astype(str)
df[RESPONSE_COLUMN] = df[RESPONSE_COLUMN].astype(str)

# INPUTS TO MODEL
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)

input_df_tmp = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
input_columns = list(input_df_tmp.columns)
input_df = preprocess.convert_input(input_df_tmp)
display(input_df.head())

In [ ]:
# OUTPUTS OF MODEL
# converts the coded columns into wide form 1/0 binary responses for every option 
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long, OUTPUT_COLUMNS, N_COLUMNS)
output_df.head()
output_columns = list(output_df.columns)

## Create Synthetic Data

* To augment our training data, produce synthetic data from the available phrases in the code list. 
* Both singular phrases and multi-response phrases are produced.
* The multi-response phrases are randomly generated according to weights associated with the non-written responses. 

In [ ]:
reload(synthetic)

In [ ]:
# create synthetic data
# this section will create synthetic data that matches a single category based on available phrases 
extra_input_df, extra_output_df = synthetic.create_single_phrase_synthetic(
    output_df, 
    input_columns,
    output_columns,
    code_df_long,
    n_per_category = N_PER_CATEGORY,
    use_given = False
)

In [ ]:
# create synthetic data
# this section will create synthetic data that matches multiple categories
mixed_input_df, mixed_output_df = synthetic.create_multi_phrase_synthetic(
    output_df,
    df_closed,
    input_columns,
    output_columns,
    code_df_long,
    N_SYNTHETIC_MIXED
)

In [ ]:
extra_input_df.head()

In [ ]:
mixed_input_df.head()

In [ ]:
chunk_size = 1000
chunks_extra_input = [extra_input_df.iloc[i:i+chunk_size] for i in range(0, len(extra_input_df), chunk_size)]
chunks_mixed_input = [mixed_input_df.iloc[i:i+chunk_size] for i in range(0, len(mixed_input_df), chunk_size)]

In [ ]:
processed_chunks_extra_input = []
n_chunks = len(chunks_extra_input)
for idx, chunk in enumerate(chunks_extra_input):
    try:
        processed_chunk = preprocess.convert_input(chunk)
        processed_chunks_extra_input.append(processed_chunk)
        print(f"Chunk {idx+1:02}/{n_chunks} processed successfully for extra_input!", end = '\r')
    except Exception as e:
        print(f"Error in chunk {idx} for extra_input: {e}")

extra_input_df = pd.concat(processed_chunks_extra_input, ignore_index=True)

In [ ]:
processed_chunks_mixed_input = []
n_chunks = len(chunks_mixed_input)
for idx, chunk in enumerate(chunks_mixed_input):
    try:
        processed_chunk = preprocess.convert_input(chunk)
        processed_chunks_mixed_input.append(processed_chunk)
        print(f"Chunk {idx+1:02}/{n_chunks} processed successfully for mixed_input!", end='\r')
    except Exception as e:
        print(f"Error in chunk {idx} for mixed_input: {e}")

mixed_input_df = pd.concat(processed_chunks_mixed_input, ignore_index=True)

In [ ]:
# Concatenate data
final_input_df = pd.concat(
    [
        #input_df, 
        extra_input_df, 
        mixed_input_df
    ], 
    ignore_index=True).drop('response', axis=1).astype(float)
final_output_df = pd.concat(
    [
        #output_df, 
        extra_output_df, 
        mixed_output_df
    ]
    , 
    ignore_index=True).drop('response', axis=1).astype(int)

In [ ]:
final_input_df.head()

In [ ]:
final_output_df.head()

In [ ]:
final_input_df.shape

In [ ]:
final_output_df.shape

## Train Model

To deal with the fact we have many possible categorical outputs, use a simple random forest model that will handle multiple outputs better than other models.

* After training, save model. 

In [ ]:
reload(model)

In [ ]:
n_estimators = 100
clf = model.create_model(final_input_df, final_output_df, n_estimators = n_estimators)

In [ ]:
model.save_model(CRED_PATH, MODEL_BASE_PATH, clf, code_df_long)

In [ ]:
clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)

## Produce Model Results

* Look at some results by hand to see if it makes sense
* Create results for entire hand-coded dataset
* Send results back to databse

In [ ]:
sentence = 'uk'
model.list_classes(sentence, code_df_long, clf, truncate_inputs=True, spellcheck=True)

In [ ]:
reload(model)

In [ ]:
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    OUTPUT_COLUMNS,
    N_COLUMNS,
    question = 'Q22',
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)

In [ ]:
results_df.head()

In [ ]:
# save back to database
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)

# for initial save of cycle 1, always replace. any subsequent inputs should be appended
connect.save_table(results_df, RESULTS_TABLE, engine, how='replace')